In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 각 날짜의 데이터를 저장할 리스트
games_data = []

for game_code in range(20190010, 20191009):
    url = f'https://statiz.sporki.com/schedule/?m=gamelogs&s_no={game_code}'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

    response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    html = response.text

    # 응답에서 데이터 추출
    soup = BeautifulSoup(html, "html.parser")

    box_divs = soup.find_all('div', class_='sh_box')
    lineup_divs = box_divs[:2]

    home_lineup = []
    away_lineup = []

    for i, div in enumerate(lineup_divs):
        a_tags = div.find_all('a')
        for a_tag in a_tags:
            if i == 0:
                home_lineup.append(a_tag.text)
            else:
                away_lineup.append(a_tag.text)

    try:
        score_divs = soup.find_all('div', class_='num')
        away_score = score_divs[0].text.split(':')[0]
        if away_score == '경기취소':
            continue
        home_score = score_divs[0].text.split(':')[1]
    except:
        continue    

    head_divs = soup.find_all('div', class_='box_head')
    home_team = head_divs[0].text.strip().split(',')[0]
    away_team = head_divs[1].text.strip().split(',')[0]

    # 날짜 데이터 추출
    date_divs = soup.find_all('div', class_='txt')
    game_date = date_divs[0].text.strip()
    # 날짜 데이터 수정
    game_date_parts = game_date.split(', ')
    game_date = int('2019' + game_date_parts[1].replace('-', ''))
    if game_date < 20190323 or game_date > 20191026:
        continue

    games_data.append([game_date, away_team, ', '.join(away_lineup), home_team, ', '.join(home_lineup), away_score, home_score])

In [14]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(games_data, columns=['Game Date', 'Away Team', 'Away Lineup', 'Home Team', 'Home Lineup', 'Away Score', 'Home Score'])
df.to_csv('match_record_2019.csv', index=False)